In [9]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import statsmodels.stats.multitest as smm

#### Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

#### Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). 

In [5]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')
normal = df[df['Diagnosis'] == 'normal']
early_neoplasia = df[df['Diagnosis'] == 'early neoplasia']
cancer = df[df['Diagnosis'] == 'cancer']

Применим t-тест для проверки гипотезы о равенстве средних в двух независимых выборках (Применим критерий для каждого гена в выборке дважды - для здоровых людей & людей на ранней стадии рака, для людей на ранней стадии рака & людей с сильно выраженными симптомами). Посчитаем количество статистически значимых отличий в генах (признаках) между выборками.

In [78]:
gens = normal.columns[2:]

alpha = 0.05

_, p_value_norm_neo = ttest_ind(normal[gens], early_neoplasia[gens], equal_var = False)
_, p_value_neo_canc = ttest_ind(early_neoplasia[gens], cancer[gens], equal_var = False)

norm_neo_cnt = sum(p_value_norm_neo < alpha)
neo_canc_cnt = sum(p_value_neo_canc < alpha)
total_cnt = norm_neo_cnt + neo_canc_cnt

print("Количество статистически значимых отличий между norm vs neo: {}".format(norm_neo_cnt))
print("Количество статистически значимых отличий между neo vs canc: {}".format(neo_canc_cnt))
print("Суммарное количество статистически значимых отличий: {}".format(total_cnt))

with open("answer1.txt", "w") as answ:
    answ.write(str(norm_neo_cnt))
    
with open("answer2.txt", "w") as answ:
    answ.write(str(neo_canc_cnt))

Количество статистически значимых отличий между norm vs neo: 1575
Количество статистически значимых отличий между neo vs canc: 3490
Суммарное количество статистически значимых отличий: 5065


Посчитаем поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку мы делаем поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

In [82]:
rej_norm_neo, pval_corr_norm_neo, _, _ = smm.multipletests(p_value_norm_neo, alpha=alpha/2, method='holm')
rej_neo_canc, pval_corr_neo_canc, _, _ = smm.multipletests(p_value_neo_canc, alpha=alpha/2, method='holm')

total_cnt = sum(rej_norm_neo) + sum(rej_neo_canc)
print("Количество статистически значимых отличий: {}".format(total_cnt)) 

Количество статистически значимых отличий: 81


Выведем количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем сделаем это с учетом практической значимости: посчитаем для каждого значимого изменения fold change и выпишем в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

**Обратите внимание, что**

* применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
* при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [83]:
def fold_change(C, T, limit=1.5):
    '''
    C - control sample
    T - treatment sample
    '''
    if T >= C:
        fc_stat = T / C
    else:
        fc_stat = -C / T
    
    return (np.abs(fc_stat) > limit), fc_stat

In [84]:
# Normal vs neoplasia
gen_normal = normal[gens].iloc[:, rej_norm_neo]
gen_neoplasia = early_neoplasia[gens].iloc[:, rej_norm_neo]

normal_neoplasia = [fold_change(norm, neopl)[0]
                    for norm, neopl in zip(gen_normal.mean(), gen_neoplasia.mean())]

norm_neo_cnt = sum(normal_neoplasia)
      
        
#Neoplasia vs cancer samples
gen_neoplasia1 = early_neoplasia[gens].iloc[:, rej_neo_canc]
gen_cancer = cancer[gens].iloc[:, rej_neo_canc]

neoplasia_cancer = [fold_change(neopl, canc)[0]
                    for neopl, canc in zip(gen_neoplasia1.mean(), gen_cancer.mean())]

neo_canc_cnt = sum(neoplasia_cancer)


print('Normal vs neoplasia samples fold change above 1.5: %d' % norm_neo_cnt)
print('Neoplasia vs cancer samples fold change above 1.5: %d' % neo_canc_cnt)

with open("answer3.txt", "w") as answ:
    answ.write(str(norm_neo_cnt))
    
with open("answer4.txt", "w") as answ:
    answ.write(str(neo_canc_cnt))

Normal vs neoplasia samples fold change above 1.5: 2
Neoplasia vs cancer samples fold change above 1.5: 77


Проделаем то же самое с использованием поправки Бенджамини-Хохберга

In [80]:
rej_norm_neo, pval_corr_norm_neo, _, _ = smm.multipletests(p_value_norm_neo, alpha=alpha/2, method='fdr_bh')
rej_neo_canc, pval_corr_neo_canc, _, _ = smm.multipletests(p_value_neo_canc, alpha=alpha/2, method='fdr_bh')

total_cnt = sum(rej_norm_neo) + sum(rej_neo_canc)
print("Количество статистически значимых отличий: {}".format(total_cnt)) 

# Normal vs neoplasia
gen_normal = normal[gens].iloc[:, rej_norm_neo]
gen_neoplasia = early_neoplasia[gens].iloc[:, rej_norm_neo]

normal_neoplasia = [fold_change(norm, neopl)[0]
                    for norm, neopl in zip(gen_normal.mean(), gen_neoplasia.mean())]

norm_neo_cnt = sum(normal_neoplasia)
      
        
#Neoplasia vs cancer samples
gen_neoplasia1 = early_neoplasia[gens].iloc[:, rej_neo_canc]
gen_cancer = cancer[gens].iloc[:, rej_neo_canc]

neoplasia_cancer = [fold_change(neopl, canc)[0]
                    for neopl, canc in zip(gen_neoplasia1.mean(), gen_cancer.mean())]

neo_gen_cnt = sum(neoplasia_cancer)


print('Normal vs neoplasia samples fold change above 1.5: %d' % norm_neo_cnt)
print('Neoplasia vs cancer samples fold change above 1.5: %d' % neo_canc_cnt)

with open("answer5.txt", "w") as answ:
    answ.write(str(norm_neo_cnt))
    
with open("answer6.txt", "w") as answ:
    answ.write(str(neo_canc_cnt))

Количество статистически значимых отличий: 836
Normal vs neoplasia samples fold change above 1.5: 4
Neoplasia vs cancer samples fold change above 1.5: 524
